# Python Intro - Final Exercise


In [1]:
import pyodbc
import pandas as pd
import numpy as np

In [2]:
### print each driver I have access to
for driver in pyodbc.drivers():
    print(driver)

SQL Server
SQL Server Native Client 11.0
ODBC Driver 11 for SQL Server
Microsoft Access Driver (*.mdb, *.accdb)
Microsoft Excel Driver (*.xls, *.xlsx, *.xlsm, *.xlsb)
Microsoft Access dBASE Driver (*.dbf, *.ndx, *.mdx)
Microsoft Access Text Driver (*.txt, *.csv)


In [3]:
### In windows, Using a ODBC DNS (predefined connection name)
conn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                      "Server=DESKTOP-6O28P02\SQLEXPRESS;"
                      "Database=COLLEGE;"
                      "Trusted_Connection=yes;")



In [4]:
### Get the students table
students = pd.read_sql("SELECT * FROM Students;",conn)

students.head()

,StudentId,FirstName,LastName,Gender
0,162.0,Elmira,Holston,F
1,154.0,Sierra,Van Vorst,F
2,28.0,Mycah,Bingham,F
3,54.0,Alia,Jones,F
4,31.0,Chanda,Fillmore,F


In [5]:
### Get the classrooms table
classrooms = pd.read_sql("SELECT * FROM Classrooms;",conn)

classrooms.head()

,CourseId,StudentId,degree
0,1.0,202.0,92.031070
1,1.0,12.0,81.175916
2,1.0,232.0,82.396587
3,1.0,123.0,62.420449
4,1.0,274.0,72.569900


In [6]:
### Get the courses table
courses = pd.read_sql("SELECT * FROM Courses;",conn)

courses.head()

,CourseId,CourseName,DepartmentID,TeacherId
0,1.0,English Begginers,1.0,15.0
1,2.0,Advanced English,1.0,15.0
2,3.0,Proffesional English,1.0,15.0
3,4.0,Mathematics 101,2.0,2.0
4,5.0,Matematics Advanced,2.0,13.0


In [7]:
### Get the departments table
departments = pd.read_sql("SELECT * FROM Departments;",conn)

departments.head()

,DepartmentId,DepartmentName
0,1.0,English
1,2.0,Science
2,3.0,Arts
3,4.0,Sport


In [8]:
### Get the teachers table
teachers = pd.read_sql("SELECT * FROM Teachers;",conn)

teachers.head()

,TeacherId,FirstName,LastName,Gender
0,1.0,Kaylea,Monzingo,F
1,2.0,Amanda,Dworkin,F
2,3.0,Jordyn,Baron,F
3,4.0,Katherine,Dibari,F
4,5.0,Heather,Metcalf,F


# Questions
 
 
## Q1. Count the number of students on each department

In [9]:
class_student = pd.merge(classrooms, students, how='inner', on='StudentId')
class_student.head()

,CourseId,StudentId,degree,FirstName,LastName,Gender
0,1.0,202.0,92.031070,Autumn,Major,F
1,4.0,202.0,73.387328,Autumn,Major,F
2,8.0,202.0,83.563932,Autumn,Major,F
3,18.0,202.0,66.109849,Autumn,Major,F
4,20.0,202.0,63.988269,Autumn,Major,F


In [10]:
len(class_student)

754

In [16]:
class_course = pd.merge(class_student, courses, how='left', on='CourseId')
class_course.head()

,CourseId,StudentId,degree,FirstName,LastName,Gender,CourseName,DepartmentID,TeacherId
0,1.0,202.0,92.031070,Autumn,Major,F,English Begginers,1.0,15.0
1,4.0,202.0,73.387328,Autumn,Major,F,Mathematics 101,2.0,2.0
2,8.0,202.0,83.563932,Autumn,Major,F,Chemistry,2.0,14.0
3,18.0,202.0,66.109849,Autumn,Major,F,R,2.0,17.0
4,20.0,202.0,63.988269,Autumn,Major,F,History of Arts,3.0,8.0


In [17]:
len(class_course)

754

In [40]:
course_department = pd.merge(class_course, departments, how='left', left_on='DepartmentID', right_on='DepartmentId')
###course_department.describe()

In [43]:
result1 = course_department.groupby('DepartmentName').agg({'StudentId':['nunique']})

In [45]:
result1.rename(columns={'StudentId':' ','nunique':'Students'}, inplace=True)
result1

,
,Students
DepartmentName,
Arts,137
English,84
Science,200
Sport,81


,StudentId
,
DepartmentName,
Arts,137
English,84
Science,200
Sport,81


## Q2. How many students have each course of the English department and the total number of students in the department?

In [48]:
## Create table with the English department data only
English_courses = course_department[course_department.DepartmentName=='English']


In [51]:
result2 = English_courses.groupby('CourseName').agg({'StudentId':['nunique']})

In [97]:
result2.rename(columns={'StudentId':'','nunique':'Students'}, inplace=True)
result2

,
,Students
CourseName,
Advanced English,24
English Begginers,32
Proffesional English,38


In [53]:
## print the total number of students at the English department
print("Total number of English students is %s" % (English_courses.StudentId.nunique()))

Total number of English students is 84


## Q3. How many small (<22 students) and large (22+ students) classrooms are needed for the Science department?

In [92]:
## Create table with science department courses (DepartmentID=2)
Science_courses = class_course[class_course.DepartmentID==2].groupby('CourseName').agg({'StudentId':['nunique']})

In [93]:
Science_courses['Classroom_size']='Large_classroom'
Science_courses.loc[(Science_courses['StudentId','nunique']<22),'Classroom_size']='Small classroom'

In [94]:
result3 = Science_courses.groupby('Classroom_size').agg({('StudentId','nunique'):['count']})

In [96]:
result3.rename(columns={'StudentId':'','nunique':'', 'count':'Number of classrooms'}, inplace=True)
result3

,
,
,Number of classrooms
Classroom_size,
Large_classroom,10
Small classroom,6


## Q4. A feminist student claims that there are more male than female in the College. Justify if the argument is correct

In [98]:
result4 = students.groupby('Gender').agg({'StudentId':['nunique']})

In [99]:
result4.rename(columns={'StudentId':'','nunique':'Number of students'}, inplace=True)
result4

,
,Number of students
Gender,
F,165
M,115


## Q5. For which courses the percentage of male/female students is over 70%?

In [185]:
class_course_2=class_course
class_course_2['Female_student']=0
class_course_2.loc[(class_course['Gender']=='F'),'Female_student']=1
gender_percentage = class_course.groupby(['CourseId','CourseName']).agg({'Female_student':['sum','count','mean']})

In [186]:
gender_percentage.columns = ['Sum','Count','Mean']
gender_percentage = gender_percentage.rename_axis(['CoureId','CourseName']).reset_index()
gender_percentage

,CoureId,CourseName,Sum,Count,Mean
0,1.0,English Begginers,20,32,0.625000
1,2.0,Advanced English,13,24,0.541667
2,3.0,Proffesional English,19,38,0.500000
3,4.0,Mathematics 101,11,18,0.611111
4,5.0,Matematics Advanced,3,10,0.300000
5,6.0,Algebra,8,12,0.666667
6,7.0,Geometry,9,14,0.642857
7,8.0,Chemistry,16,24,0.666667
8,9.0,Physics,6,12,0.500000
9,10.0,Biology,19,36,0.527778


In [249]:
gender_percentage_over_70 = gender_percentage[gender_percentage.Mean>0.7]
del gender_percentage_over_70 ['Sum']
del gender_percentage_over_70 ['Count']
##gender_percentage_over_70.rename(columns={'Mean':'Female Percent'}, inplace=True)
gender_percentage_over_70

,CoureId,CourseName,Mean
21,22.0,Sculpture,0.708333
26,29.0,Tenis,0.722222


## Q6. For each department, how many students passed with a grades over 80?

In [255]:
result6 = course_department[course_department['degree']>80].\
groupby('DepartmentName').agg({'StudentId':['nunique']})
result6.rename(columns={'StudentId':'','nunique':'Students over 80'}, inplace=True)
result6

,
,Students over 80
DepartmentName,
Arts,61
English,40
Science,122
Sport,39


## Q7. For each department, how many students passed with a grades under 60?

In [256]:
result7 = course_department[course_department['degree']<60].\
groupby('DepartmentName').agg({'StudentId':['nunique']})
result7.rename(columns={'StudentId':'','nunique':'Students under 60'}, inplace=True)
result7

,
,Students under 60
DepartmentName,
Arts,3
English,2
Science,7
Sport,1


## Q8. Rate the teachers by their average student's grades (in descending order).

In [227]:
teacher_course = pd.merge(class_course, teachers, how='left', on='TeacherId')
teacher_course['Teacher'] = teacher_course['FirstName_y'] + " " +teacher_course['LastName_y']

In [228]:
teacher_course.head()

,CourseId,StudentId,degree,FirstName_x,LastName_x,Gender_x,CourseName,DepartmentID,TeacherId,Female_student,FirstName_y,LastName_y,Gender_y,Teacher
0,1.0,202.0,92.031070,Autumn,Major,F,English Begginers,1.0,15.0,1,Conor,Mckittrick,M,Conor Mckittrick
1,4.0,202.0,73.387328,Autumn,Major,F,Mathematics 101,2.0,2.0,1,Amanda,Dworkin,F,Amanda Dworkin
2,8.0,202.0,83.563932,Autumn,Major,F,Chemistry,2.0,14.0,1,Jeffrey,Batzel,M,Jeffrey Batzel
3,18.0,202.0,66.109849,Autumn,Major,F,R,2.0,17.0,1,Tanner,Fitzsimons,M,Tanner Fitzsimons
4,20.0,202.0,63.988269,Autumn,Major,F,History of Arts,3.0,8.0,1,Caroline,Knuesel,F,Caroline Knuesel


In [229]:
teacher_grade = teacher_course.groupby(['Teacher']).agg({'degree':['mean']})
teacher_grade.columns = ['Avg degrees']
teacher_grade = teacher_grade.rename_axis(['Teacher']).reset_index()
teacher_grade.sort_values(ascending=False,by='Avg degrees')

,Teacher,Avg degrees
5,Darick Hess,81.432703
9,Jordyn Baron,80.729494
7,Heather Metcalf,80.440242
21,Zachary Healy,79.365867
12,Kaylea Monzingo,79.365537
4,Daniel Bland,78.908281
13,Keegan Price,78.814631
3,Conor Mckittrick,78.699492
16,Maxwell Romkee,78.673509
20,Terran Beaty,78.495479


## Q9. Create a dataframe showing the courses, departments they are associated with, the teacher in each course, and the number of students enrolled in the course (for each course, department and teacher show the names).

In [245]:
course_students = pd.merge(teacher_course, departments, how='left', left_on='DepartmentID', right_on='DepartmentId')
course_students.head()

,CourseId,StudentId,degree,FirstName_x,LastName_x,Gender_x,CourseName,DepartmentID,TeacherId,Female_student,FirstName_y,LastName_y,Gender_y,Teacher,DepartmentId,DepartmentName
0,1.0,202.0,92.031070,Autumn,Major,F,English Begginers,1.0,15.0,1,Conor,Mckittrick,M,Conor Mckittrick,1.0,English
1,4.0,202.0,73.387328,Autumn,Major,F,Mathematics 101,2.0,2.0,1,Amanda,Dworkin,F,Amanda Dworkin,2.0,Science
2,8.0,202.0,83.563932,Autumn,Major,F,Chemistry,2.0,14.0,1,Jeffrey,Batzel,M,Jeffrey Batzel,2.0,Science
3,18.0,202.0,66.109849,Autumn,Major,F,R,2.0,17.0,1,Tanner,Fitzsimons,M,Tanner Fitzsimons,2.0,Science
4,20.0,202.0,63.988269,Autumn,Major,F,History of Arts,3.0,8.0,1,Caroline,Knuesel,F,Caroline Knuesel,3.0,Arts


In [246]:
result9 = pd.DataFrame([course_students.CourseId,
                        course_students.CourseName,
                        course_students.DepartmentName,
                        course_students.Teacher,
                        course_students.StudentId]).transpose()

In [247]:
result9 = result9.groupby(['CourseId','CourseName'],as_index=False).agg({'DepartmentName':['min'],
                           'Teacher':['min'],
                           'StudentId':['nunique']})

result9.rename(columns={'StudentId':'Number of students','min':'','nunique':''}, inplace=True)
result9

,CourseId,CourseName,DepartmentName,Teacher,Number of students
,,,,,
0,1.0,English Begginers,English,Conor Mckittrick,32
1,2.0,Advanced English,English,Conor Mckittrick,24
2,3.0,Proffesional English,English,Conor Mckittrick,38
3,4.0,Mathematics 101,Science,Amanda Dworkin,18
4,5.0,Matematics Advanced,Science,Alexander Watts,10
5,6.0,Algebra,Science,Alexander Watts,12
6,7.0,Geometry,Science,Alexander Watts,13
7,8.0,Chemistry,Science,Jeffrey Batzel,23
8,9.0,Physics,Science,Alexander Watts,12


## Q10. Create a dataframe showing the students, the number of courses they take, the average of the grades per class, and their overall average (for each student show the student name).

In [276]:
course_department['English']=np.nan
course_department['Arts']=np.nan
course_department['Science']=np.nan
course_department['Sports']=np.nan

In [277]:
course_department.describe()

,CourseId,StudentId,degree,DepartmentID,TeacherId,DepartmentId,English,Arts,Science,Sports
count,754.000000,754.000000,754.000000,754.000000,754.000000,754.000000,0.0,0.0,0.0,0.0
mean,15.687003,139.814324,78.206629,2.376658,13.267905,2.376658,NaN,NaN,NaN,NaN
std,8.528534,82.819329,8.507021,0.860085,6.306459,0.860085,NaN,NaN,NaN,NaN
min,1.000000,1.000000,49.236544,1.000000,1.000000,1.000000,NaN,NaN,NaN,NaN
25%,10.000000,66.000000,72.769352,2.000000,9.000000,2.000000,NaN,NaN,NaN,NaN
50%,16.000000,141.000000,78.370188,2.000000,15.000000,2.000000,NaN,NaN,NaN,NaN
75%,23.000000,213.000000,83.672210,3.000000,19.000000,3.000000,NaN,NaN,NaN,NaN
max,31.000000,280.000000,100.000000,4.000000,24.000000,4.000000,NaN,NaN,NaN,NaN


In [278]:
course_department.loc[(course_department['DepartmentName']=='English'),'English']=course_department['degree']
course_department.loc[(course_department['DepartmentName']=='Arts'),'Arts']=course_department['degree']
course_department.loc[(course_department['DepartmentName']=='Science'),'Science']=course_department['degree']
course_department.loc[(course_department['DepartmentName']=='Sport'),'Sports']=course_department['degree']

In [279]:
result10=pd.DataFrame(course_department)

In [280]:
result10=result10[['StudentId','FirstName','LastName','CourseId','English','Arts','Science','Sports']]
result10 = result10.rename({"CourseId": "Courses"},axis=1)
result10.head()

,StudentId,FirstName,LastName,Courses,English,Arts,Science,Sports
0,202.0,Autumn,Major,1.0,92.03107,NaN,NaN,NaN
1,202.0,Autumn,Major,4.0,NaN,NaN,73.387328,NaN
2,202.0,Autumn,Major,8.0,NaN,NaN,83.563932,NaN
3,202.0,Autumn,Major,18.0,NaN,NaN,66.109849,NaN
4,202.0,Autumn,Major,20.0,NaN,63.988269,NaN,NaN


In [281]:
result10 = result10.groupby(['StudentId'],as_index=False).agg({'FirstName':[min],
                   'LastName':['min'],
                   'Courses':['nunique'],
                   'English':['mean'],
                   'Arts':['mean'],
                   'Science':['mean'],
                   'Sports':['mean']}) 

In [282]:
result10['Avarage_grade']=np.nan
result10['Avarage_grade']=result10.iloc[:,[4,5,6,7]].mean(axis=1)

In [283]:
result10.rename(columns={'min':'','mean':'','nunique':''}, inplace=True)
result10.head(14)

,StudentId,FirstName,LastName,Courses,English,Arts,Science,Sports,Avarage_grade
,,,,,,,,,
0,1.0,Alexandria,Heller,4,NaN,86.677535,79.557508,NaN,83.117522
1,2.0,Cassie,Liska,5,NaN,79.542738,78.867115,NaN,79.204927
2,3.0,Hayley,Frank,3,83.972245,NaN,76.204427,NaN,80.088336
3,4.0,Maxwell,Kotch,3,70.929801,69.356144,74.157066,NaN,71.481003
4,5.0,Dylan,Boehm,3,NaN,64.494845,100.000000,65.661106,76.718650
5,6.0,Chelsea,Vuolo,2,NaN,68.799129,74.358535,NaN,71.578832
6,7.0,Katrina,Evers,4,NaN,NaN,73.325298,NaN,73.325298
7,8.0,Sean,Hasenack,3,89.073972,96.216926,79.304200,NaN,88.198366
8,9.0,Madalyn,Miller,2,NaN,NaN,86.802830,NaN,86.802830


In [285]:
len(result10)

256